In [1]:
!uv add pydantic-ai

Resolved 197 packages in 2.53s                                       
Prepared 63 packages in 1.05s                                            
Uninstalled 1 package in 10ms
░░░░░░░░░░░░░░░░░░░░ [0/63] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 63 packages in 809ms                              
 + ag-ui-protocol==0.1.9
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.1
 + aiosignal==1.4.0
 + anthropic==0.71.0
 + argcomplete==3.6.3
 + boto3==1.40.57
 + botocore==1.40.57
 + cachetools==6.2.1
 + cohere==5.19.0
 + docstring-parser==0.17.0
 + eval-type-backport==0.2.2
 + fastavro==1.12.1
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.9.0
 + genai-

In [4]:
import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [6]:
from minsearch import Index

index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [7]:
from typing import Any, Dict, List, TypedDict

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str

def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - title (str): The title of the matched document.
            - description (str): A short description of the document.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )

In [8]:
file_index = {}

for item in parsed_data:
    filename = item['filename']
    content = item['content']
    file_index[filename] = content

In [9]:
len(file_index)

95

In [10]:
def read_file(filename: str) -> str:
    """
    Retrieve the contents of a file from the file index if it exists.

    Args:
        filename (str): The name of the file to read.

    Returns:
        str: The file's contents if found, otherwise an error message 
        indicating that the file does not exist.
    """
    if filename in file_index:
        return file_index[filename]
    return "File doesn't exist"

In [11]:
from pydantic_ai import Agent

In [12]:
documentation_agent_instructions = """
You are an assistant that helps improve and generate high-quality documentation for the project.

You have access to the following tools:
- search — Use this to explore topics in depth. Make multiple search calls if needed to gather comprehensive information.
- read_file — Use this when code snippets are missing or when you need to retrieve the full content of a file for context.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

When answering a question:
1. Provide file references for all source materials.  
   Use this format:  
   [{filename}](https://github.com/evidentlyai/docs/blob/main/{filename})
2. If the topic is covered in multiple documents, cite all relevant sources.
3. Include code examples whenever they clarify or demonstrate the concept.
4. Be concise, accurate, and helpful — focus on clarity and usability for developers.
5. If documentation is missing or unclear, infer from context and note that explicitly.

Example Citation:
See the full implementation in [metrics/api_reference.md](https://github.com/evidentlyai/docs/blob/main/metrics/api_reference.md).
""".strip()

In [13]:
documentation_agent = Agent(
    name='documentation_agent',
    instructions=documentation_agent_instructions,
    tools=[search, read_file],
    model='openai:gpt-4o-mini'
)

In [16]:
results= await documentation_agent.run(
    user_prompt='how do i run llm as a judge evals',
    message_history = results.all_messages()
)

In [17]:
print(results.output)

To run a Large Language Model (LLM) as a judge for evaluations, you can follow the tutorial that outlines a practical example using Python. Here's a brief overview of the steps involved:

### Tutorial Overview
1. **Install Required Libraries**:
   Install the Evidently library which is used to run evaluations.
   ```bash
   pip install evidently
   ```

2. **Import Modules and Set Up API**:
   You will need to import necessary libraries and set your OpenAI API key.
   ```python
   import os
   os.environ["OPENAI_API_KEY"] = "YOUR_KEY"
   ```

3. **Create an Evaluation Dataset**:
   Prepare a dataset for evaluation which includes:
   - Questions
   - Target responses (approved answers)
   - New responses (what the system generates)
   - Manual labels for evaluation
   
   This can be done using pandas to create a DataFrame.

4. **Design the LLM Evaluator**:
   Create and run an LLM evaluator prompt. The LLM will evaluate the responses against the target responses.
   ```python
   from e

In [18]:
results.new_messages

<bound method AgentRunResult.new_messages of AgentRunResult(output='To run a Large Language Model (LLM) as a judge for evaluations, you can follow the tutorial that outlines a practical example using Python. Here\'s a brief overview of the steps involved:\n\n### Tutorial Overview\n1. **Install Required Libraries**:\n   Install the Evidently library which is used to run evaluations.\n   ```bash\n   pip install evidently\n   ```\n\n2. **Import Modules and Set Up API**:\n   You will need to import necessary libraries and set your OpenAI API key.\n   ```python\n   import os\n   os.environ["OPENAI_API_KEY"] = "YOUR_KEY"\n   ```\n\n3. **Create an Evaluation Dataset**:\n   Prepare a dataset for evaluation which includes:\n   - Questions\n   - Target responses (approved answers)\n   - New responses (what the system generates)\n   - Manual labels for evaluation\n   \n   This can be done using pandas to create a DataFrame.\n\n4. **Design the LLM Evaluator**:\n   Create and run an LLM evaluator p

In [20]:
for message in results.new_messages():
    print(message.kind)
    for part in message.parts:
        print(part.part_kind)

    print()

request
user-prompt

response
tool-call

request
tool-return

response
text



In [21]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import PydanticAIRunner

In [22]:
runner = PydanticAIRunner(
    chat_interface=IPythonChatInterface(),
    agent=documentation_agent
)

In [23]:
await runner.run()

You: how do i run llm as a judge


You: stop


Chat ended.
